In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
saved_model = pickle.load(open('news_categorization_model.pickle', 'rb'))

In [3]:
clf = saved_model[0]
title_vectorizer = saved_model[1]
text_vectorizer = saved_model[2]
one_hot_encoder = saved_model[3]
scaler = saved_model[4]
feature_name = saved_model[5]
numerical_feature = saved_model[6]
ordinal_feature = saved_model[7]
nominal_feature = saved_model[8]

In [4]:
feature_name

['title', 'body_text']

In [5]:
title = 'กรีนพีซ แฉรัฐหมกเม็ดรายชื่อเกษตรกร'
body_text = '''กรุงเทพฯ-7 ก.ย.47  ผอ.กรีนพีซ ตั้งข้อสังเกต รายชื่อเกษตรกรผู้รับเมล็ดพันธุ์มะละกอจากสำนักวิจัยฯ ขอนแก่น อาจไม่ใช่เอกสารตัวจริง เนื่องจากลงรายมือชื่อไม่เหมือนกัน   
 "ในรายชื่อของเอกสาร  คนที่ไปซื้อต้องเขียนชื่อที่อยู่และเซ็นชื่อด้วยลายมือของตนเอง  แต่เท่าที่รู้ตอนนี้ซึ่งผมยังไม่ยืนยัน  รายชื่อที่ส่งให้คณะกรรมการสิทธิฯ เป็นรายชื่อที่คัดมาแล้ว เพราะเป็นลายมือของคนคนเดียว มันไม่ได้เป็นเอกสารตัวจริง"  นายจิรากรณ์ คชเสนี  ผอ.กรีนพีซ เอเชียตะวันออกเฉียงใต้ระบุ   
ก่อนหน้านี้ รัฐมนตรีว่าการกระทรวงเกษตรและสหกรณ์  ให้สัมภาษณ์ว่า ยินดีเปิดเผยรายชื่อเกษตรกรที่ได้รับแจกเมล็ดพันธุ์มะละกอแขกดำพันธุ์ท่าพระ จำนวน 2,300 กว่ารายชื่อ ตามที่กรรมการสิทธิมนุษยชนแห่งชาติร้องขอ เพื่อแสดงความบริสุทธิ์ใจให้ตรวจสอบว่า มีมะละกอปนเปื้อนจีเอ็มโอหลุดรอดจากแปลงทดลองหรือไม่   
นายจิรากรณ์  กล่าวว่า  อยากให้กรมวิชาการเกษตรและกระทรวงเกษตรและสหกรณ์ยอมรับความจริงเรื่องการปนเปื้อนของมะละกอจีเอ็มโอที่สถานีวิจัยฯ ท่าพระ จ.ขอนแก่น  และแสดงความรับผิดชอบโดยเปิดเผยข้อมูลรวมทั้งรายชื่อผู้ซื้อเมล็ดพันธุ์และกล้ามะละกอที่เป็นจริง   
นอกจากนี้นายจิรากรณ์ยังเรียกร้องให้มีการติดตามไปตรวจสอบและทำลายมะละกอที่ปนเปื้อนจีเอ็มโอในไร่นาเกษตรกรที่ซื้อเมล็ดพันธุ์โดยจ่ายค่าชดเชยให้เกษตรกรด้วย  รวมทั้งเปิดเผยสัญญาบันทึกความเข้าใจ (เอ็มโอยู) ที่กรมวิชาการเกษตรทำกับมหาวิทยาลัยคอร์แนลหรือบริษัทข้ามชาติอื่นๆ ด้วย     
  รายงานโดย : มุทิตา เชื้อชั่ง
ศูนย์ข่าวประชาไท [color] 
'''

In [6]:
incoming_data = pd.DataFrame(np.array([[title, body_text]]), columns=feature_name)

incoming_data

,title,body_text
0,กรีนพีซ แฉรัฐหมกเม็ดรายชื่อเกษตรกร,กรุงเทพฯ-7 ก.ย.47 ผอ.กรีนพีซ ตั้งข้อสังเกต รา...


### title - Count Vectorization

In [7]:
title_corpus_incoming = incoming_data['title'].tolist()
title_cnt_vec_incoming = title_vectorizer.transform(title_corpus_incoming).toarray()

In [8]:
title_cnt_vec_feature_name = [
    "cnt_title_" + feature for feature in title_vectorizer.get_feature_names()
]

In [9]:
incoming_data[title_cnt_vec_feature_name] = title_cnt_vec_incoming
incoming_data.drop('title', axis=1, inplace=True)

### text - Count Vectorization

In [10]:
text_corpus_incoming = incoming_data['body_text'].tolist()
text_cnt_vec_incoming = text_vectorizer.transform(text_corpus_incoming).toarray()

In [11]:
text_cnt_vec_feature_name = [
    "cnt_text_" + feature for feature in text_vectorizer.get_feature_names()
]

In [12]:
incoming_data[text_cnt_vec_feature_name] = text_cnt_vec_incoming
incoming_data.drop('body_text', axis=1, inplace=True)

### ===== Ordinal Encoding & One Hot Encoding =====

In [13]:
# print('numerical_feature :', numerical_feature)
# print('ordinal_feature :', ordinal_feature)
# print('nominal_feature :', nominal_feature)

### ### Ordinal Encoding

### One Hot Encoding

In [14]:
one_hot_feature = []
for i, feature in enumerate(nominal_feature):
    for cate in one_hot_encoder.categories_[i]:
        one_hot_feature_name = str(feature) + '_' + str(cate)
        one_hot_feature.append(one_hot_feature_name)
        
incoming_data[one_hot_feature] = one_hot_encoder.transform(incoming_data[nominal_feature])
incoming_data.drop(nominal_feature, axis=1, inplace=True)

In [15]:
incoming_data

,cnt_title_,cnt_title_,cnt_title_!,"cnt_title_""","cnt_title_""สม",cnt_title_#,cnt_title_%,cnt_title_',cnt_title_(,cnt_title_),...,cnt_text_ไหม,cnt_text_ไหว,cnt_text_ไอ,cnt_text_ๆ,cnt_text_์,cnt_text_๒,cnt_text_๕,cnt_text_’,cnt_text_“,cnt_text_”
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,3,0,0,0,0,0


### Feature Scaling

In [16]:
incoming_data_scaled = scaler.transform(incoming_data)

### Prediction

In [17]:
predicted_target = clf.predict(incoming_data_scaled)

In [18]:
predicted_target

array(['environment'], dtype=object)